In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import libraries 

In [ ]:
import dataclasses
import json
import os
import pathlib

import numpy as np
import torch
from jinja2 import Environment, FileSystemLoader
from src.four_dim_srda.config.config_u_net_vit_v2 import UNetVitVer02ExpConfig
from src.four_dim_srda.config.experiment_config import LossConfig, TrainConfig
from src.four_dim_srda.data.dataloader import DataloaderConfig
from src.four_dim_srda.data.dataset_making_obs import DatasetMakingObsConfig
from src.four_dim_srda.models.neural_nets.u_net_vit_v2 import UNetVitVer02Config

# Define constants

In [ ]:
ROOT_DIR = pathlib.Path(os.environ["PYTHONPATH"]).parent.resolve()

experiment_name = "experiment7"
experiment_cfg_name = UNetVitVer02ExpConfig

model_name = "UNetVitVer02"
model_cfg_name = UNetVitVer02Config

DATA_DIR = (
    f"{ROOT_DIR}/data/four_dim_srda/{experiment_name}/cfd_simulation/qg_model/max_and_min_value"
)

CFG_DIR = f"{ROOT_DIR}/python/configs/four_dim_srda/{experiment_name}/perform_4D_SRDA/{model_name}"
os.makedirs(CFG_DIR, exist_ok=True)

In [ ]:
SCRIPT_TRAIN_DIR = f"{ROOT_DIR}/python/bash/four_dim_srda/{experiment_name}/DL_train_on_es/{model_name}"
SCRIPT_EVAL_DIR = f"{ROOT_DIR}/python/bash/four_dim_srda/{experiment_name}/DL_evaluate_on_es/{model_name}"

os.makedirs(SCRIPT_TRAIN_DIR, exist_ok=True)
os.makedirs(SCRIPT_EVAL_DIR, exist_ok=True)

In [ ]:
data = np.load(f"{DATA_DIR}/values.npz")
min_val, max_val = torch.tensor(data["min_val"]), torch.tensor(data["max_val"])

margin = 0.1
_min = min_val - (max_val - min_val) * margin
_max = max_val + (max_val - min_val) * margin

pv_min, pv_max = round(torch.min(_min).item(), 2), round(torch.max(_max).item(), 2)

In [ ]:
pv_min, pv_max

# Define methods

In [ ]:
def make_cfg_and_script(
    *,
    cfg,
    experiment_name,
    model_name,
    cfg_dir,
    script_train_dir,
    script_eval_dir,
):
    # パラメータ
    bea = int(cfg.dataset_config.beta_dist_alpha)
    bed = int(cfg.dataset_config.beta_dist_beta)
    dspe = cfg.dataset_config.data_size_per_epoch
    nsls = cfg.dataset_config.num_searched_lr_states
    ogx = f"{cfg.dataset_config.obs_grid_interval_x:02}"
    ogy = f"{cfg.dataset_config.obs_grid_interval_y:02}"

    n3drb = cfg.model_config.num_3d_resnet_block
    nvb = cfg.model_config.num_vit_block
    bias = int(cfg.model_config.bias)
    vits = int(cfg.model_config.use_vit_skip_connect)

    bs = cfg.dataloader_config.batch_size

    lr = f"{cfg.train_config.lr:.0e}"

    # Configの保存
    cfg_name = f"bea{bea}_bed{bed}_dspe{dspe}_nsls{nsls}_ogx{ogx}_ogy{ogy}_n3drb{n3drb}_nvb{nvb}_bias{bias}_vits{vits}_bs{bs}_lr{lr}.yml"
    cfg.save(pathlib.Path(f"{cfg_dir}/{cfg_name}"))

    #
    context = {
        #
        "experiment_name": experiment_name,
        #
        "model_name": model_name,
        "bea": bea,
        "bed": bed,
        "dspe": dspe,
        "nsls": nsls,
        "ogx": ogx,
        "ogy": ogy,
        #
        "n3drb": n3drb,
        "nvb": nvb,
        "bias": bias,
        "vits": vits,
        #
        "bs": bs,
        "lr": lr,
    }

    # Jinja2環境の設定
    env = Environment(
        loader=FileSystemLoader(f"{script_train_dir}"), trim_blocks=True, lstrip_blocks=True
    )

    # Trainingテンプレートの読み込み & レンダリング
    template = env.get_template("template.sh.j2")
    rendered_script = template.render(context)

    # スクリプト生成
    output_file = f"{script_train_dir}/train_ddp_bea{bea}_bed{bed}_dspe{dspe}_nsls{nsls}_ogx{ogx}_ogy{ogy}_n3drb{n3drb}_nvb{nvb}_bias{bias}_vits{vits}_bs{bs}_lr{lr}.sh"
    with open(output_file, "w") as f:
        f.write(rendered_script)

    # 実行権限を付与
    os.chmod(output_file, 0o740)

    # Jinja2環境の設定
    env = Environment(
        loader=FileSystemLoader(f"{script_eval_dir}"), trim_blocks=True, lstrip_blocks=True
    )

    # Testテンプレートの読み込み & レンダリング
    template = env.get_template("template.sh.j2")
    rendered_script = template.render(context)

    # スクリプト生成
    output_file = f"{script_eval_dir}/bea{bea}_bed{bed}_dspe{dspe}_nsls{nsls}_ogx{ogx}_ogy{ogy}_n3drb{n3drb}_nvb{nvb}_bias{bias}_vits{vits}_bs{bs}_lr{lr}.sh"
    with open(output_file, "w") as f:
        f.write(rendered_script)

    # 実行権限を付与
    os.chmod(output_file, 0o740)

    # Testテンプレートの読み込み & レンダリング
    template = env.get_template("template_using_narrow_jet.sh.j2")
    rendered_script = template.render(context)

    # スクリプト生成
    output_file = f"{script_eval_dir}/bea{bea}_bed{bed}_dspe{dspe}_nsls{nsls}_ogx{ogx}_ogy{ogy}_n3drb{n3drb}_nvb{nvb}_bias{bias}_vits{vits}_bs{bs}_lr{lr}_using_narrow_jet.sh"
    with open(output_file, "w") as f:
        f.write(rendered_script)

    # 実行権限を付与
    os.chmod(output_file, 0o740)

# Make

In [ ]:
# データを作り直した場合はmax_start_time_indexの変更を忘れないこと
# input_sequence_lengthの値の変更も忘れないこと
# hr_sequence_lengthの値の変更も忘れないこと

model_config = UNetVitVer02Config(
    model_name=model_name,
    bias=True,
    input_channels=1,
    feat_channels_0=128,
    feat_channels_1=128,
    feat_channels_2=128,
    feat_channels_3=128,
    encoder_output_channels=128,
    input_sequence_length=3,
    hr_sequence_length=9,
    num_3d_resnet_block=3,
    num_vit_block=4,
    encoder_activation_type="silu",
    decoder_activation_type="silu",
    other_activation_type="silu",
    encoder_norm_type="layer",
    decoder_norm_type="layer",
    other_norm_type="layer",
    patch_size_x=1,
    patch_size_y=1,
    emb_dim=512,
    n_head=8,
    hidden_dim=512 * 2,
    dropout=0.2,
    hr_x_size=128,
    hr_y_size=64,
    hr_z_size=16,
    lr_x_size=32,
    lr_y_size=16,
    lr_z_size=4,
    use_vit_skip_connect=False,
)

dataset_config = DatasetMakingObsConfig(
    data_dirs=[],
    use_ground_truth_clipping=None,
    lr_and_obs_time_interval=4,
    max_start_time_index=792,
    beta_dist_alpha=2.0,
    beta_dist_beta=2.0,
    data_size_per_epoch=360,
    is_future_obs_missing=True,
    max_ensemble_number=20,
    missing_value=0.0,
    num_searched_lr_states=100,
    nx_hr=128,
    ny_hr=65,
    nz_hr=16,
    pv_max=pv_max,
    pv_min=pv_min,
    obs_grid_interval_x=8,
    obs_grid_interval_y=8,
    obs_noise_std=0.022,
    use_mixup=True,
    use_observation=True,
)

dataloader_config = DataloaderConfig(
    dataset_name="DatasetMakingObs",
    batch_size=12,
    data_dir_name=f"{experiment_name}/cfd_simulation/qg_model/hr_and_lr_pv",
    num_workers=2,
    train_valid_test_ratios=[0.7, 0.2, 0.1],
    seed=42,
)

train_config = TrainConfig(
    early_stopping_patience=51,
    num_epochs=400,
    lr=0.0001,
    seed=42,
    use_zero=True,
)

loss_config = LossConfig(name="L1")

cfg = experiment_cfg_name(
    model_config=model_config,
    dataset_config=dataset_config,
    dataloader_config=dataloader_config,
    train_config=train_config,
    loss_config=loss_config,
)


make_cfg_and_script(
    cfg=cfg,
    experiment_name=experiment_name,
    model_name=model_name,
    cfg_dir=CFG_DIR,
    script_train_dir=SCRIPT_TRAIN_DIR,
    script_eval_dir=SCRIPT_EVAL_DIR,
)

In [ ]:
# # tuning beta

# betas = [3, 6, 9, 12]

# for beta in betas:
#     dataset_config.beta_dist_beta = beta

#     #
#     make_cfg_and_script(
#         cfg=cfg,
#         experiment_name=experiment_name,
#         model_name=model_name,
#         cfg_dir=CFG_DIR,
#         script_train_dir=SCRIPT_TRAIN_DIR,
#         script_eval_dir=SCRIPT_EVAL_DIR,
#     )

In [ ]:
# # tuning lr
# lrs = [1e-5, 3e-5, 5e-5, 8e-5, 1e-4, 2e-4]
# for lr in lrs:
#     #
#     train_config.lr = lr

#     make_cfg_and_script(
#         cfg=cfg,
#         experiment_name=experiment_name,
#         model_name=model_name,
#         cfg_dir=CFG_DIR,
#         script_train_dir=SCRIPT_TRAIN_DIR,
#         script_eval_dir=SCRIPT_EVAL_DIR,
#     )